Try them here [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yupbank/tree_experiment/blob/master/experiments/regression_experiment_example.ipynb)

In [1]:
import sys
sys.path.append('../')
import scipy
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.datasets import load_boston
import scipy.stats as stats
import itertools
from bsz.utils import (
    bsplitz_method,
    fast_skewness_improvements,
    fast_variance_improvements,
)
from bsz.cube_to_zonotope import enumerate_all_points


In [2]:
def report_regression_comparison(
    mean_encoding,
    generators,
    d,
    measure=fast_skewness_improvements,
    measure_name="skewness",
):
    potentials = np.cumsum(generators[np.argsort(mean_encoding)], axis=0)
    res = measure(potentials, d)

    all_candidates = enumerate_all_points(generators)
    new_res = measure(all_candidates, d)

    another_potentials, _ = bsplitz_method(generators)
    similar_res = measure(another_potentials, d)

    print("best %s improvement from mean encoding" % measure_name, np.max(res))
    print("best %s improvement from greedy enumerating" % measure_name, np.max(new_res))
    print(
        "best %s improvement from smart enumerating" % measure_name, np.max(similar_res)
    )

In [3]:
    data = load_boston()
    x = data["data"][:, 8]
    y = data["target"]
    d = np.array([y.sum(), (y ** 2).sum(), y.shape[0]])
    hx = OneHotEncoder(sparse=False, categories="auto").fit_transform(x[:, np.newaxis])

    generators = np.vstack([hx.T.dot(y), hx.T.dot(y ** 2), hx.sum(axis=0)]).T

    mean_encoding = generators[:, 0] / generators[:, -1]

    report_regression_comparison(mean_encoding, generators, d)
    report_regression_comparison(
        mean_encoding, generators, d, fast_variance_improvements, "variance"
    )

best skewness improvement from mean encoding 100.75924213358303
best skewness improvement from greedy enumerating 103.60639381029978
best skewness improvement from smart enumerating 103.60639381029978
best variance improvement from mean encoding 14.918864090329691
best variance improvement from greedy enumerating 14.918864090329691
best variance improvement from smart enumerating 14.918864090329691


../bsz/utils.py:24: RuntimeWarning: divide by zero encountered in true_divide
  e = 3 * b / a * c / a - 2 * (b / a) ** 3
../bsz/utils.py:24: RuntimeWarning: invalid value encountered in multiply
  e = 3 * b / a * c / a - 2 * (b / a) ** 3
../bsz/utils.py:24: RuntimeWarning: invalid value encountered in true_divide
  e = 3 * b / a * c / a - 2 * (b / a) ** 3
../bsz/utils.py:33: RuntimeWarning: divide by zero encountered in true_divide
  e = np.square(b / a)
../bsz/utils.py:33: RuntimeWarning: invalid value encountered in true_divide
  e = np.square(b / a)
